# Accuracy and Error Types

To start, reload the Naive Bayes classifier from 18.05.  Note how little code is actually required to generate the model.

In [42]:
import scipy
import sklearn

import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sns

from sklearn.naive_bayes import BernoulliNB
from sklearn.metrics import confusion_matrix # New for this module

In [134]:
# Grab and process the raw data.
data_path = ("https://raw.githubusercontent.com/Thinkful-Ed/data-201-resources/"
             "master/sms_spam_collection/SMSSpamCollection"
            )
sms_raw = pd.read_csv(data_path, delimiter= '\t', header=None)
sms_raw.columns = ['spam', 'message']

# Enumerate our spammy keywords.
keywords = ['click', 'offer', 'winner', 'buy', 'free', 'cash', 'urgent']

for key in keywords:
    sms_raw[str(key)] = sms_raw.message.str.contains(
        ' ' + str(key) + ' ',
        case=False
)

sms_raw['allcaps'] = sms_raw.message.str.isupper()
sms_raw['spam'] = (sms_raw['spam'] == 'spam')
data = sms_raw[keywords + ['allcaps']]
target = sms_raw['spam']

bnb = BernoulliNB()
y_pred = bnb.fit(data, target).predict(data)

## Success Rate

Now that the model has returned its predictions, the first thing to note is what data is comparable for model evaluation, the _target_ and _y_pred_ variables.  _Target_ is the actual outcomes, wether something was spam or ham.  The _y_pred_ is the predicted outcomes from the classifier.  Both are ordered arrays with the results from each row of the dataframe.  When they agree the model was successfully able to predict wether a given message was spam or ham.  When they disagree the model was incorrect.

The most basic measure of success is how often the model was correct.  This is called the accuracy.  It's a metric that you've seen before, in the previous challenge but translated from a count to a percentage.

In [135]:
# target is the actual outcomes from the dataset
actual_spam = target.sum() # = 747 

# y_pred is the predicted outcomes from the classifier
bayes_predicted = y_pred.sum() # = 253

# Number of messages spam & ham
all_messages = data.shape[0] # = 5572

# Number of messages the classifier guessed correctly
bayes_correct = data.shape[0]-(target != y_pred).sum() # = 4968

# Number of message incorrectly guessed
missclassified = (target != y_pred).sum() # = 604

# Calculate the accuracy of the classifier
accuracy = bayes_correct/all_messages

print(f"{accuracy*100:,.2f}% accuracy.  Bayes Correct: {bayes_correct}, Total messages: {all_messages}, Actual Spam: {actual_spam},  Bayes Predicted Spam: {bayes_predicted}, Bayes Misclassified: {missclassified}")

89.16% accuracy.  Bayes Correct: 4968, Total messages: 5572, Actual Spam: 747,  Bayes Predicted Spam: 253, Bayes Misclassified: 604


Success rate is a popular way to evaluate a model, and what most people get excited about when discussing a model.  However, success rate is usually not sufficient.  

First, not all errors are created equal.  Consider the spam example, letting in a spam message is not nearly as egregious as throwing out a real (and potentially important) message.  Knowing more about the kinds of errors you're generating can be incredibly useful.  

Second, understanding how the model is failing can be key to improving it.  If a certain outcome is not being predicted accurately you may want to focus on entineering more features to identify that outcome.

## Consusion Matrix

The next level of analysis of the classifier is often called the Confusion Matrix.  This is a matrix that show the count of each possible permutation of target and prediction.  In the spam example, the matrix will show the counts for when a message was ham and the model predicted ham, when a message was ham and the model predicted spam, when a message was spam and the model predicted ham, and finally, when a message was spam and the model predicted spam.  SKLearn has a built in confusion matrix function, use it below.


In [136]:
confusion_matrix(target, y_pred)

array([[4770,   55],
       [ 549,  198]], dtype=int64)

For this confusion matrix, the columns are prediction and the rows are actual.  The majority of the error is coming from the times where the model failed to identify a spam message.  549 of 604 errors are from failing to identify spam.  So the model needs to get better at identifying spam messages.  

Before moving on to iterate on the model, here are some key terms that you may encounter when thinking about this kind of matrix.

Assume the goal is to identify spam (rather than identify ham).

When discussing errors in a binary classifier (where there are only two outcomes) we're generally referring to two kinds of errors.  A __false positive__ is when the model identifies something as spam that is not.  In this case there are 55 of these.  This is also called a "Type I Error" or "false alarm".

A __false negative__ is therefore when the model mistakenly identifies something as not spam when it is.  There were 549 of these.  This is also called a "Type II Error" or a "miss".

__Sensitivity__ is the percentage of positives correctly identified, for the model: 198/747 or 27%.  This shows how good the model is catching positives, or how sensitive the model is to identifying positives.

__Specificity__ is the opposite, the percentage of negatives correctly identified, 4770/4825 or 99%.

This demonstrates that the model is not great at identifying spam, though the model does label ham quite accurately.  These terms are important and you need to have a firm grasp on their meaning.  

## Drill

It's worth calculating these with code so that you fully understand how these statistics work, so here is your task for the cell below.  Manually generate (__meaning don't use SKLearn function__) your own confusion matrix and print it along with the sensitivty and specificity.

In [260]:
confusion_matrix = dict(
    false_positives=((~target)&(y_pred)).sum(),
    false_negatives=((target)&(~y_pred)).sum(),
    y_pred_correct_pos=((target)&(y_pred)).sum(),
    y_pred_correct_neg=((~target)&(~y_pred)).sum(),
    target_pos=(target).sum(),
    target_neg=(~target).sum(),
    sensitivity=(((target)&(y_pred)).sum()/(target).sum())*100,     
    specificity=(((~target)&(~y_pred)).sum()/(~target).sum())*100, 
    total_errors=(target != y_pred).sum()
)

print(f"""
\tThe model produced {confusion_matrix["total_errors"]} errors, {confusion_matrix["false_negatives"]} of these were from the model failing to identify spam.  The model produced {confusion_matrix["false_positives"]} False Positives or "Type I Errors".  

There were {confusion_matrix["false_negatives"]} False Negatives or "Type II Errors".  The model correctly identified {confusion_matrix["y_pred_correct_pos"]}/{confusion_matrix["target_pos"]} ham messages which translates to a Sensitivity of {confusion_matrix["sensitivity"]:,.4f}%.

The model did a much better job of identifying spam with {confusion_matrix["y_pred_correct_neg"]}/{confusion_matrix["target_neg"]} spam messages identified.  The model's Specificity is {confusion_matrix["specificity"]:,.4f}%.
""")


	The model produced 604 errors, 549 of these were from the model failing to identify spam.  The model produced 55 False Positives or "Type I Errors".  

There were 549 False Negatives or "Type II Errors".  The model correctly identified 198/747 ham messages which translates to a Sensitivity of 26.5060%.

The model did a much better job of identifying spam with 4770/4825 spam messages identified.  The model's Specificity is 98.8601%.

